In [1]:
import keras, os, pickle, re, sklearn, string, tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras.layers import Embedding
from keras.optimizers import Adadelta, adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils.vis_utils  import plot_model
from sklearn import preprocessing
from keras import optimizers

print('Keras version: \t\t%s' % keras.__version__)
print('Scikit version: \t%s' % sklearn.__version__)
print('TensorFlow version: \t%s' % tensorflow.__version__)


Using TensorFlow backend.


Keras version: 		2.0.9
Scikit version: 	0.19.1
TensorFlow version: 	1.4.1


In [2]:
import nltk
from nltk.corpus import stopwords
from tika import parser
from sklearn.metrics import classification_report



In [3]:
def LoadData_tech4():
    label_nontech=[]
    i=0
    for foldername in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level4/tech/software"):
        for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level4/tech/software/"+foldername):
    
            try:
                print (i, foldername, file)
                parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level4/tech/software/"+foldername+"/"+file)

                resume_contents=clean_doc(parsedPDF['content'])
            #Data = resume_contents.encode('utf-8')    
                label_nontech.append((resume_contents,foldername))
            except UnicodeEncodeError:
                print ('Unicode error:', file)
            i=i+1
    #print (label_resume)
    return(label_nontech)
        

In [4]:
#Preprocessing
def clean_doc(doc):
    """
    Cleaning a document by several methods:
        - Lowercase
        - Removing whitespaces
        - Removing stopwords
        - Removing punctuations
        
    """
    stop_words = set(stopwords.words('english'))
    
    # Lowercase
    doc = doc.lower()
    # Removing multiple whitespaces
    doc = re.sub(r"\?", " \? ", doc)
    # Remove numbers
    #doc = re.sub(r"[0-9]+", "", doc)
    # Split in tokens
    tokens = doc.split()
    # Remove Stopwords
    tokens = [w for w in tokens if not w in stop_words]
    # Remove punctuation
    tokens = [w.translate(str.maketrans('', '', string.punctuation)) for w in tokens]
    # Tokens with less then two characters will be ignored
    #tokens = [word for word in tokens if len(word) > 1]
    return ' '.join(tokens)

In [5]:
label_tech=LoadData_tech4()

0 Other AbhishekThakurProfile.pdf
1 Other RobertAndersonProfile.pdf
2 Other WesScharfProfile.pdf
3 Other HardikRathoreProfile.pdf
4 Other VincentSProfile.pdf
5 Other DenysKliuchProfile.pdf
6 Other KavitaMishraProfile.pdf
7 Other NuthanPereiraProfile.pdf
8 Other ShantanuPrakashProfile.pdf
9 Other MariosMichailidisProfile.pdf
10 Other ThriveniT KProfile.pdf
11 Other agyapalsinghProfile.pdf
12 Other IvanRiveraProfile.pdf
13 Other BaskaryabaseProfile.pdf
14 Other NikitaPuzereyProfile.pdf
15 Other Jaideep SinghBhatiProfile.pdf
16 Other JamesJohnProfile.pdf
17 Other RobinSharmaProfile(1).pdf
18 Other RaghuveerSinghProfile.pdf
19 Other VitalyKiktenkoProfile.pdf
20 Other OleksiiBlyzniukProfile.pdf
21 Other DeepakRamesh Profile.pdf
22 Other GinaPooleProfile.pdf
23 Other Dr. DeepikaSharmaProfile.pdf
24 Other WrijuGhoshProfile.pdf
25 Other Guruprasad GProfile.pdf
26 Other EricMcQuigganProfile.pdf
27 Other SerhiiYeryfaProfile.pdf
28 Other JihinRajuProfile.pdf
29 Other AshifIqbalProfile.pdf
30 Othe

215 Tester HIMANSHUSHARMAProfile.pdf
216 Tester Rohini Rattan.doc
217 Tester BERI_KARTHIK_Hyderabad___Secunderabad_2.01_yrs.docx
218 Tester Sarumathi_Subramani_Bengaluru___Bangalore_5.04_yrs.doc
219 Tester Anusha A.S.doc
220 Tester Raja_Ram_Mohan_Roy_S_Chennai_4.02_yrs.pdf
221 Tester Anirban_Bandyopadhyay_Bengaluru___Bangalore_5.01_yrs.docx
222 Tester Prathibha__Chilakaraju_Hyderabad___Secunderabad_7.00_yrs.doc
223 Tester 56550931_Hyderabad___Secunderabad_5.09_yrs.doc
224 Tester Arindam_Changdar_Bengaluru___Bangalore_0.06_yrs.docx
225 Tester Sowjanya___Hyderabad___Secunderabad_7.00_yrs.docx
226 Tester Harshada_Bhalerao_Pune_1.11_yrs.docx
227 Tester Nikhil RajSinghProfile.pdf
228 Tester Sheelu-Resume.doc
229 Tester Murgesh_Alias_Hari_Muniyandi_Chennai_6.07_yrs.doc
230 Tester SenthilKumar_Sundharamoorthy_Singapore_7.06_yrs.doc
231 Tester Deepa_S_Chennai_4.06_yrs.docx
232 Tester VINOTH KUMAR SEVAGAMURTHY.doc
233 Tester Resume Calpine.doc
234 Tester Saravanakumar_Jayaraman_Chennai_6.05_yrs

391 developer Gaurav_Balyan_Noida_4.11_yrs.docx
392 developer srinivas[2_1].docx
393 developer Arun_Pareek_Hyderabad___Secunderabad_4.10_yrs.pdf
394 developer SanuRaj[3_6].pdf
395 developer Soumyaranjan[2_5].docx
396 developer pallavi_bhole_Mumbai_1.10_yrs.doc
397 developer VamsiMullapudi[3_2].pdf
398 developer Sagar[7_0].docx
399 developer ANUSHAK[3_1].pdf
400 developer VimalKesavan[3_11].docx
401 developer SibinA[0_5].pdf
402 developer SVINOTHKUMAR[4_8].docx
403 developer Nikhil__Pune_3.00_yrs.doc
404 developer RukminiKR[4_4].pdf
405 developer Anil_Kumar_Nomula_Hyderabad___Secunderabad_0.06_yrs.pdf
406 developer Lekshmi Sadanandan.doc
407 developer AjithCPurushothaman[3_4].docx
408 developer Sri_Rao__Malaysia_3.00_yrs.doc
409 developer Vineet_kumar_Dubey_Mumbai_4.10_yrs.pdf
410 developer 60659278_Bengaluru___Bangalore_4.05_yrs.docx
411 developer shaik_karimulla__Bengaluru_-_Bangalore_0.00_yrs.docx
412 developer vijayaramreddy[2_8].doc
413 developer RAKESH[4_0].doc
414 developer Sreej

In [6]:
len(label_tech)

450

In [7]:

jdres_train_labels=[]
jdres_train_data=[]

for row in label_tech:
    jdres_train_data.append(row[0])
    jdres_train_labels.append(row[1])
#print(jdres_train_data)
#print(len(jdres_train_data))

In [8]:
print(len(jdres_train_data))
print(len(jdres_train_labels))

450
450


In [9]:
df1 = pd.DataFrame(jdres_train_data,columns=['filecontent'])
df2 = pd.DataFrame(jdres_train_labels,columns=['label'])

In [10]:
#print (df2)

In [11]:
#print(jdres_train_data[42])

In [12]:
encoder = preprocessing.LabelEncoder()
encoder.fit(jdres_train_labels)
encoded_labels = encoder.transform(jdres_train_labels)
# convert integers to dummy variables (i.e. one hot encoded)
labels = to_categorical(encoded_labels,num_classes=3)
print(encoded_labels)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2]


In [13]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(jdres_train_data)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(jdres_train_data)
vocab_size

23000

In [14]:
# pad documents to a max length of1000 words
max_length = 500
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[   37    22  2263 ...,     0     0     0]
 [ 1017   462   490 ...,  5592   202 10579]
 [   37    22  4109 ...,     0     0     0]
 ..., 
 [22912   588 22913 ...,    42  3208    90]
 [  310     5   234 ...,    40 10511   220]
 [ 7236  7237  4098 ...,     0     0     0]]


In [15]:
print('Shape of data tensor:', padded_docs.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (450, 500)
Shape of label tensor: (450, 3)


In [16]:
indices = np.arange(padded_docs.shape[0])
np.random.shuffle(indices)
padded_docs = padded_docs[indices]
labels = labels[indices]

In [17]:
#loading glove

embeddings_index = dict()
f = open('/home/shabna/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [18]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [19]:
#create model

from keras.layers import Activation, Input, Dense, Flatten, Dropout, Embedding
from keras.layers import Conv1D, MaxPooling1D,GlobalMaxPooling1D
from keras.models import Model

In [20]:
#from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [88]:
def create_model():
    model = Sequential()
    e = Embedding(vocab_size,100, weights=[embedding_matrix], input_length=500, trainable='true')
    model.add(e)
    model.add(Conv1D(128,5, activation='relu',name='l1'))
    model.add(MaxPooling1D(pool_size=5,name='l2'))
    model.add(Conv1D(128, 5, activation='relu',name='l3'))
    model.add(GlobalMaxPooling1D(name='l4'))
   
    model.add(Dense(9, activation='sigmoid',name='l5'))
    model.add(Dropout(0.2))
    #model.add(Flatten())
    model.add(Dense(3, activation='softmax',name='l6'))
    #adam = optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    return model

In [89]:
#model = Model(embedding_matrix, preds)
model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 100)          2300000   
_________________________________________________________________
l1 (Conv1D)                  (None, 496, 128)          64128     
_________________________________________________________________
l2 (MaxPooling1D)            (None, 99, 128)           0         
_________________________________________________________________
l3 (Conv1D)                  (None, 95, 128)           82048     
_________________________________________________________________
l4 (GlobalMaxPooling1D)      (None, 128)               0         
_________________________________________________________________
l5 (Dense)                   (None, 9)                 1161      
_________________________________________________________________
dropout_2 (Dropout)          (None, 9)                 0         
__________

In [ ]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

param_grid = dict(learn_rate=learn_rate, momentum=momentum, batch_size=batch_size, epochs=epochs,optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
    grid_result = grid.fit(X_train, y_train,validation_data=(X_val,y_val))
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [90]:
from keras.callbacks import ModelCheckpoint

In [91]:
from keras.callbacks import Callback

In [92]:
filepath="/home/shabna/Desktop/example_codes/weights_4_1.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only="True", mode="max")
callbacks_list = [checkpoint]

In [93]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)

XX_train, X_test, yy_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=42)

i=1
for train_index, test_index in kf.split(XX_train):
    #print("TRAIN:", train_index, "TEST:", test_index)
    print(i,":num of fold")

    X_train, X_val = XX_train[train_index], XX_train[test_index]
    y_train, y_val = yy_train[train_index], yy_train[test_index]
    
    history = model.fit(X_train, y_train, validation_data=(X_val,y_val) ,epochs=50, batch_size=100, verbose=1, callbacks=callbacks_list)
   
    i=i+1
    

1 :num of fold
Train on 240 samples, validate on 120 samples
Epoch 1/50
240/240 [==============================] - 8s 35ms/step - loss: 1.1598 - acc: 0.3125 - val_loss: 1.0657 - val_acc: 0.3583
Epoch 2/50
240/240 [==============================] - 6s 23ms/step - loss: 1.0435 - acc: 0.4333 - val_loss: 1.0007 - val_acc: 0.4417
Epoch 3/50
240/240 [==============================] - 6s 24ms/step - loss: 0.9268 - acc: 0.6625 - val_loss: 0.9210 - val_acc: 0.6917
Epoch 4/50
240/240 [==============================] - 6s 24ms/step - loss: 0.8615 - acc: 0.6750 - val_loss: 0.8096 - val_acc: 0.7833
Epoch 5/50
240/240 [==============================] - 6s 23ms/step - loss: 0.7521 - acc: 0.7958 - val_loss: 0.7665 - val_acc: 0.8000
Epoch 6/50
240/240 [==============================] - 6s 24ms/step - loss: 0.6855 - acc: 0.7917 - val_loss: 0.6997 - val_acc: 0.8417
Epoch 7/50
240/240 [==============================] - 5s 22ms/step - loss: 0.6304 - acc: 0.8583 - val_loss: 0.6450 - val_acc: 0.8333
Epoch 8/

Epoch 30/50
240/240 [==============================] - 6s 23ms/step - loss: 0.2543 - acc: 0.9875 - val_loss: 0.3287 - val_acc: 0.9333
Epoch 31/50
240/240 [==============================] - 5s 23ms/step - loss: 0.2679 - acc: 0.9750 - val_loss: 0.3273 - val_acc: 0.9333
Epoch 32/50
240/240 [==============================] - 5s 22ms/step - loss: 0.2623 - acc: 0.9750 - val_loss: 0.3249 - val_acc: 0.9333
Epoch 33/50
240/240 [==============================] - 5s 22ms/step - loss: 0.2652 - acc: 0.9625 - val_loss: 0.3222 - val_acc: 0.9417
Epoch 34/50
240/240 [==============================] - 5s 23ms/step - loss: 0.2370 - acc: 0.9875 - val_loss: 0.3187 - val_acc: 0.9417
Epoch 35/50
240/240 [==============================] - 5s 23ms/step - loss: 0.2616 - acc: 0.9542 - val_loss: 0.3156 - val_acc: 0.9417
Epoch 36/50
240/240 [==============================] - 6s 26ms/step - loss: 0.2459 - acc: 0.9833 - val_loss: 0.3134 - val_acc: 0.9333
Epoch 37/50
240/240 [==============================] - 8s 31ms

Epoch 45/50
240/240 [==============================] - 6s 23ms/step - loss: 0.1589 - acc: 0.9917 - val_loss: 0.1144 - val_acc: 1.0000
Epoch 46/50
240/240 [==============================] - 6s 24ms/step - loss: 0.1546 - acc: 0.9875 - val_loss: 0.1138 - val_acc: 0.9917
Epoch 47/50
240/240 [==============================] - 6s 25ms/step - loss: 0.1429 - acc: 0.9958 - val_loss: 0.1133 - val_acc: 0.9917
Epoch 48/50
240/240 [==============================] - 6s 25ms/step - loss: 0.1694 - acc: 0.9875 - val_loss: 0.1126 - val_acc: 0.9917
Epoch 49/50
240/240 [==============================] - 5s 22ms/step - loss: 0.1611 - acc: 0.9833 - val_loss: 0.1118 - val_acc: 0.9917
Epoch 50/50
240/240 [==============================] - 5s 22ms/step - loss: 0.1614 - acc: 0.9958 - val_loss: 0.1106 - val_acc: 0.9917
3 :num of fold
Train on 240 samples, validate on 120 samples
Epoch 1/50
240/240 [==============================] - 6s 23ms/step - loss: 0.1617 - acc: 0.9875 - val_loss: 0.1014 - val_acc: 1.0000
Ep

Epoch 28/50
240/240 [==============================] - 11s 45ms/step - loss: 0.1157 - acc: 1.0000 - val_loss: 0.0798 - val_acc: 1.0000
Epoch 29/50
240/240 [==============================] - 10s 44ms/step - loss: 0.1312 - acc: 0.9917 - val_loss: 0.0792 - val_acc: 1.0000
Epoch 30/50
240/240 [==============================] - 11s 46ms/step - loss: 0.1369 - acc: 0.9958 - val_loss: 0.0785 - val_acc: 1.0000
Epoch 31/50
240/240 [==============================] - 11s 46ms/step - loss: 0.1250 - acc: 0.9833 - val_loss: 0.0779 - val_acc: 1.0000
Epoch 32/50
240/240 [==============================] - 11s 45ms/step - loss: 0.1312 - acc: 0.9958 - val_loss: 0.0772 - val_acc: 1.0000
Epoch 33/50
240/240 [==============================] - 11s 45ms/step - loss: 0.1156 - acc: 0.9792 - val_loss: 0.0766 - val_acc: 1.0000
Epoch 34/50
240/240 [==============================] - 11s 44ms/step - loss: 0.1202 - acc: 0.9792 - val_loss: 0.0760 - val_acc: 1.0000
Epoch 35/50
240/240 [==============================] - 

In [94]:
model=load_model("weights_4_1.best.hdf5")


In [31]:
plot_model(model, to_file = 'models.png')

In [95]:
y_train

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.

In [96]:
from sklearn.metrics import classification_report, confusion_matrix

In [97]:
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred,axis=1)

In [98]:
y_pred = model.predict_classes(X_test)

90/90 [==============================] - 1s 6ms/step


In [99]:
p = model.predict_proba(X_test)

90/90 [==============================] - 1s 6ms/step


In [100]:
model.evaluate(X_test,y_test)

90/90 [==============================] - 1s 8ms/step


[0.30263475179672239, 0.92222222354676986]

In [101]:
model.evaluate(X_train,y_train)

240/240 [==============================] - 1s 6ms/step


[0.21760587592919667, 0.97916666666666663]

In [102]:
labels_index = {'Other':0 ,'Tester':1,'Developer':2}
target_name = [t for t in labels_index.keys()]

In [103]:
print(classification_report(np.argmax(y_test,axis=1),y_pred, target_names=target_name))

             precision    recall  f1-score   support

      Other       0.93      0.93      0.93        28
     Tester       0.91      0.97      0.94        31
  Developer       0.93      0.87      0.90        31

avg / total       0.92      0.92      0.92        90



In [104]:
print(confusion_matrix(np.argmax(y_test,axis=1),y_pred))

[[26  0  2]
 [ 1 30  0]
 [ 1  3 27]]


In [105]:
from numpy import argmax
for i, item in enumerate (X_test):
    item= item.reshape(1, 500)
    #print (item.shape)
    classes=model.predict(item)
    p=(argmax(classes),argmax(y_test[i]))
    #print(p)
    if (argmax(classes)!=argmax(y_test[i])):
        print(p)


(0, 2)
(1, 2)
(1, 2)
(0, 1)
(1, 2)
(2, 0)
(2, 0)


In [106]:
X_test.shape


(90, 500)

In [107]:
X_train.shape

(240, 500)

In [108]:
print(X_test)

[[  293 10060  2298 ...,     0     0     0]
 [   37    22  2653 ...,     0     0     0]
 [   37    22  6869 ...,     0     0     0]
 ..., 
 [22848 10495   957 ...,     0     0     0]
 [   37    22  5876 ...,     0     0     0]
 [12043    42  4985 ...,  6000   202 12072]]


In [118]:
#predictions

parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level4/tech/software/developer/Sanjid_T.pdf")


contents=[clean_doc(parsedPDF['content'])]
            
print(contents)

['indeed resume sanjid kochi kerala sanjidt5jhxindeedemailcom  919744125676 part challenging learning environment put best creative energies competitive approach henceforth contribute pioneering fields computers information technologies education mca nehru college management 2013 2015 bca ajk college arts  science 2010 2013 skills java c j2ee sql certifications java j2ee august 2015 february 2016 additional information technical skills languages c c vb java net area interest java technologies netbeans ms visual studio dream weaver scripting languages html java script database system ms sql server ms access operating systems windows family project title judiciary supporting system organization name netgenes duration 6 months environment java db server mysql role programmer academic project abstraction judiciary supporting system project online website district  subordinate courts country upgradation infrastructure supreme court high courtsit initiated provide efficient timebound citizen

In [119]:
def prpInput(pp):
    tx = Tokenizer()
    tx.fit_on_texts(pp)
    vocab_size = len(tx.word_index)+1
    encoded_doc = tx.texts_to_sequences(pp)
    max_length = 500
    padded_docs = pad_sequences(encoded_doc, maxlen=max_length, padding='post')
    return padded_docs

In [120]:
mat=prpInput(contents)


In [121]:
prd = model.predict(mat)


In [122]:
#labels_index = {'marketing':0 ,'hr':1, 'seo':2, 'other':3}
'''rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)'''
def result(prd,contents):
    y_classes = prd.argmax(axis=-1)
    print(len(y_classes))
    lx=[]
    rev_lable_index = {0:'Developer' , 1: 'Other', 2: 'Tester'}
    for idx,lb in enumerate(y_classes):
        lx.append([contents[idx],rev_lable_index[lb]])
    return lx

In [123]:
result(prd,contents)

1


[['indeed resume sanjid kochi kerala sanjidt5jhxindeedemailcom  919744125676 part challenging learning environment put best creative energies competitive approach henceforth contribute pioneering fields computers information technologies education mca nehru college management 2013 2015 bca ajk college arts  science 2010 2013 skills java c j2ee sql certifications java j2ee august 2015 february 2016 additional information technical skills languages c c vb java net area interest java technologies netbeans ms visual studio dream weaver scripting languages html java script database system ms sql server ms access operating systems windows family project title judiciary supporting system organization name netgenes duration 6 months environment java db server mysql role programmer academic project abstraction judiciary supporting system project online website district  subordinate courts country upgradation infrastructure supreme court high courtsit initiated provide efficient timebound citize

In [124]:

pd.DataFrame(result(prd,contents),columns=['File Content','Label'])

1


,File Content,Label
0,indeed resume sanjid kochi kerala sanjidt5jhxi...,Other


In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predictions


In [ ]:
def test_load():
    i=0
    test=[]
    for file in os.listdir("/home/shabna/Desktop/example_codes/new_sample/level3/testmix/"):
        print (i, file)
        parsedPDF = parser.from_file("/home/shabna/Desktop/example_codes/new_sample/level3/testmix/"+file)
        content = clean_doc(parsedPDF['content'])
        test.append(content)
        #mat=prpInput(contents)
        #prd = model.predict(mat)
        #print(result(prd,contents))
        #pd.DataFrame(result(prd,contents),columns=['File Content','Label'])
        i=i+1
        
    return test
testinput = test_load()

In [ ]:
h=prpInput(testinput)

In [ ]:
h

In [ ]:
prdi = model.predict(h)

In [ ]:
'''rev_lable_index = {}
for key in labels_index:
    rev_lable_index[labels_index[key]] = key
print(rev_lable_index)'''
def result(prdi,testinput):
    y_classes = prdi.argmax(axis=-1)
    print("num of target items:",len(y_classes))
    lx=[]
    rev_lable_index= {0: 'marketing', 1: 'hr', 2: 'seo', 3: 'other'}
    for idx,lb in enumerate(y_classes):
        lx.append([testinput[idx],rev_lable_index[lb]])
    return lx

In [ ]:
result(prdi,testinput)

In [ ]:
import pandas as pd
pd.DataFrame(result(prdi,testinput),columns=['File Content','Label'])